In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from kernel_svm import *
from kernel_functions import *

In [2]:
train = pd.read_csv('../data/Xtr.csv', sep = ",", header = None)
trainOutput = pd.read_csv('../data/Ytr.csv', sep = ",")
test = pd.read_csv('../data/Xte.csv', sep = ",", header = None)

In [3]:
train.drop(train.columns[len(train.columns)-1], axis=1, inplace=True)
test.drop(test.columns[len(test.columns)-1], axis=1, inplace=True)

In [200]:
class multiclassSVM:
    
    def __init__(self, K=None, kernel_fun=None, C=300, **kwargs):
        self.kernel_fun = kernel_fun
        self.parameters = kwargs
        self.C = C
        self.K = K
        self.models = {}
        self.classes = None

    def fit(self, X_train, y_train):
        
        classes = np.unique(y_train)
        self.classes = classes
        for l1 in range(len(classes)):
            for l2 in range(l1+1,len(classes)):
                self.models[classes[l1],classes[l2]] = KernelSVM(K=self.K, kernel_fun=self.kernel_fun, C=self.C)
                X_train_local = X_train[(y_train == classes[l1]) + (y_train == classes[l2])]
                y_train_local = y_train[(y_train == classes[l1]) + (y_train == classes[l2])]
                y_train_local = 2*(y_train_local == classes[l1]) - 1
                self.models[classes[l1],classes[l2]].fit(X_train_local, y_train_local)
        
        
    def predict(self, X_test):
        
        ensemble_predictions = np.empty([len(X_test), 0])
        for k in self.models.keys():
            local_prediction = self.models[k].predict(X_test)
            ensemble_predictions = np.hstack((ensemble_predictions, np.array([k[0]*(local_prediction == 1) + k[1]*(local_prediction == -1)]).T))
            
        sum_array = np.array([np.sum(ensemble_predictions == c, axis = 1) for c in self.classes]).T
        final_predictions = self.classes[np.argmax(sum_array, axis=1)]
            
        return final_predictions, ensemble_predictions, sum_array
    

In [191]:
length = 1000

ttrain = train[:length]
ttrainOutput = trainOutput[:length]['Prediction']
ttest = test[:length//2]

In [192]:
mSVM = multiclassSVM(kernel_fun = linear_kernel)

In [193]:
mSVM.fit(ttrain, ttrainOutput)

/home/marc/anaconda3/lib/python3.5/site-packages/pandas/computation/expressions.py:181: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


0
     pcost       dcost       gap    pres   dres
 0:  1.5952e+05 -6.6849e+05  8e+05  5e-17  5e-14
 1:  2.8136e+04 -5.0886e+04  8e+04  2e-16  6e-14
 2:  4.5480e+03 -7.7033e+03  1e+04  2e-16  3e-14
 3:  5.0489e+01 -2.6854e+03  3e+03  2e-16  1e-14
 4: -7.7524e+02 -1.0794e+03  3e+02  2e-16  5e-15
 5: -8.8800e+02 -9.3131e+02  4e+01  2e-16  2e-15
 6: -9.0150e+02 -9.0730e+02  6e+00  2e-16  7e-16
 7: -9.0261e+02 -9.0315e+02  5e-01  2e-16  3e-16
 8: -9.0264e+02 -9.0265e+02  1e-02  2e-16  1e-16
 9: -9.0264e+02 -9.0264e+02  1e-04  2e-16  9e-17
Optimal solution found.
0
     pcost       dcost       gap    pres   dres
 0:  1.3281e+05 -4.8195e+05  6e+05  5e-17  5e-14
 1:  2.3473e+04 -3.8926e+04  6e+04  2e-16  6e-14
 2:  4.0153e+03 -5.9271e+03  1e+04  2e-16  3e-14
 3:  4.8414e+02 -1.7584e+03  2e+03  2e-16  9e-15
 4: -1.9675e+02 -4.4819e+02  3e+02  2e-16  5e-15
 5: -2.9001e+02 -3.2582e+02  4e+01  2e-16  2e-15
 6: -3.0119e+02 -3.0599e+02  5e+00  2e-16  8e-16
 7: -3.0212e+02 -3.0257e+02  5e-01  2e-16  

In [194]:
# for k, model in mSVM.models.items():
#     print(k, model.support_vectors_)

In [195]:
final_predictions, ensemble_predictions, sum_array = mSVM.predict(ttest)

In [196]:
ens_pred[0]

array([ 5.,  4.,  1.,  4.,  5.,  2.,  6.,  0.,  4.,  8.,  1.,  3.,  2.,
        0.,  5.,  1.,  6.,  2.,  1.,  3.,  0.,  7.,  0.,  2.,  4.,  1.,
        3.,  2.,  1.,  4.,  0.,  3.,  2.,  6.,  3.,  5.,  2.,  3.,  0.,
        1.,  1.,  0.,  0.,  7.,  0.])

In [197]:
sum_array[0]

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [199]:
len(final_predictions)

500